<h1>Intro</h1>
On June 17, 1983 The Police released "Synchronicity," an album which twice interrupted Michael Jackson's "Thriller" at the top of the Billboard charts. Sychronicity was my favorite album when it was released, and remains one of my favorites today.
<p>
This project looks at the staying power of this 40-year-old record. Minimally, I'll look at record sales. But I hope to uncover insight of deeper engagement which might predict longer-term durability: how often are lyrics discussed, how many musicians trying to learn these songs?

<h1>Questions</h1>
<ol>
<li>Which tracks have shown users discussing lyrics? How has this changed over time?
<li>Which songs have been covered? How has this changed over time?
    <li>How have record sales/followers for Synchronicity compared with more recent albums by former members of The Police?
</ol>

<h1>Covers- Web Scraping Second Hand Songs</h1>
This could've been done by API, but I'm web scraping to build my skills

To run this with other albums:
<ul>
    <li>swap URLs in the first step
    <li>update data cleaning to correct the first year
    <li>export with a unique name
</ul>

In [1]:
#Synchronicity(The Police)
#url_album = "https://secondhandsongs.com/release/413"
#release_year = 1983
#bandname = "police" #for file export

#Business as Usual (Men at Work)
url_album = "https://secondhandsongs.com/release/28748"
release_year = "1981"
bandname = "men_at_work" #for file export

#Thriller (Michael Jackson)
# url_album = "https://secondhandsongs.com/release/273"
# release_year = "1982"
# bandname = "jackson" #for file export

#Can’t Slow Down (Lionel Ritchie)
# url_album = "https://secondhandsongs.com/release/2306"
# release_year = "1983"
# bandname = "lionel" #for file export

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
url = url_album
http_response = requests.get(url)
html = http_response.text
soup_songlist = BeautifulSoup(html)

<h3>select the list of songs, then create a list of corresponding URLs</h3>

In [4]:
list_song_urls = []
#loop all html links in the striped table, and grab urls
for html_link in soup_songlist.select(".field-title a"):
    list_song_urls.append("https://secondhandsongs.com"+str(html_link["href"]))

#remove the header row
list_song_urls.pop(0)

#remove any submissions that aren't yet verified
for song_url in list_song_urls:
    if song_url.find("submission") != -1:
        list_song_urls.remove(song_url)

list_song_urls

['https://secondhandsongs.com/performance/876580',
 'https://secondhandsongs.com/performance/876602',
 'https://secondhandsongs.com/performance/72501',
 'https://secondhandsongs.com/performance/876591',
 'https://secondhandsongs.com/performance/256690']

<h1>For each song, count the number of covers per year</h1>
would also like to look at song name, written by, language<br>
but for now, to keep it simple

In [5]:
#create an empty data frame
df_covers = pd.DataFrame ({'song':[],'year':[]})
df_covers

Empty DataFrame
Columns: [song, year]
Index: []

In [6]:
#put it all together
for song_url in list_song_urls:
    http_response = requests.get(song_url)
    html = http_response.text
    soup_covers = BeautifulSoup(html)
    song_title = soup_covers.select('.entity-title a')[0].text
    df_next_song = pd.DataFrame([{
        "song": song_title,
        "year": next_date.text[-4:] if next_date.text != "Release date " else ""
    } for next_date in soup_covers.select(".field-date")])
    df_covers=pd.concat([df_covers,df_next_song])

<h3>Manually retrieve "Down Under" by Men at Work</h3>
<br>this song was missing from the album, perhaps because it was separately released as a single

In [7]:
#"Down Under" hack. This song was missing from Men at Work, because it was also released as a single
# df_covers = pd.DataFrame ({'song':[],'year':[]})
# df_covers

# song_url = "https://secondhandsongs.com/performance/60901"
# http_response = requests.get(song_url)
# html = http_response.text
# soup_covers = BeautifulSoup(html)
# song_title = soup_covers.select('.entity-title a')[0].text
# df_next_song = pd.DataFrame([{
#     "song": song_title,
#     "year": next_date.text[-4:] if next_date.text != "Release date " else ""
# } for next_date in soup_covers.select(".field-date")])
# df_covers=pd.concat([df_covers,df_next_song])
# bandname = "men_at_work"

<h3>clean up data</h3>
See https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
for methodology in calculating a 'decades' column using numpy

In [8]:
#remove blank years
df_covers=df_covers.drop(df_covers.query("year==''").index)
df_covers["year"] = df_covers["year"].astype(int)

In [9]:
#calculate decade
conditions = [
    (df_covers['year'] <=1992),
     (df_covers['year'] <=2002),
    (df_covers['year'] <=2012),
    (df_covers['year'] <=2022)
]
values = [1,2,3,4]
df_covers['decade'] = np.select(conditions,values)
df_covers

song  year  decade
1              Be Good Johnny  1981       1
2              Be Good Johnny  2003       3
3              Be Good Johnny  2005       3
5              Be Good Johnny  2022       4
1             Down by the Sea  1981       1
2             Down by the Sea  2005       3
1   I Can See It in Your Eyes  1981       1
2   I Can See It in Your Eyes  2000       2
3   I Can See It in Your Eyes  2010       3
1                 Underground  1981       1
2                 Underground  2005       3
1          Who Can It Be Now?  1981       1
2          Who Can It Be Now?  1982       1
3          Who Can It Be Now?  1983       1
5          Who Can It Be Now?  1996       2
6          Who Can It Be Now?  2003       3
7          Who Can It Be Now?  2004       3
8          Who Can It Be Now?  2005       3
9          Who Can It Be Now?  2005       3
10         Who Can It Be Now?  2011       3
11         Who Can It Be Now?  2011       3
12         Who Can It Be Now?  2011       3
13         Who Can It Be Now?  2013       4
14         Who Can It Be Now?  2014       4
15         Who Can It Be Now?  2014       4
16         Who Can It Be Now?  2014       4
17         Who Can It Be Now?  2014       4
18         Who Can It Be Now?  2015       4
19         Who Can It Be Now?  2016       4
20         Who Can It Be Now?  2018       4
21         Who Can It Be Now?  2018       4
22         Who Can It Be Now?  2019       4
23         Who Can It Be Now?  2021       4
25         Who Can It Be Now?  2017       4
26         Who Can It Be Now?  2022       4
27         Who Can It Be Now?  2022       4
28         Who Can It Be Now?  2022       4
30         Who Can It Be Now?  2019       4

<h3>tabulate by year</h3>

In [10]:
df_covers_count = df_covers.groupby("song").value_counts().to_frame().reset_index()
df_covers_count

song  year  decade  count
0              Be Good Johnny  2003       3      1
1              Be Good Johnny  1981       1      1
2              Be Good Johnny  2022       4      1
3              Be Good Johnny  2005       3      1
4             Down by the Sea  2005       3      1
5             Down by the Sea  1981       1      1
6   I Can See It in Your Eyes  2010       3      1
7   I Can See It in Your Eyes  2000       2      1
8   I Can See It in Your Eyes  1981       1      1
9                 Underground  2005       3      1
10                Underground  1981       1      1
11         Who Can It Be Now?  2014       4      4
12         Who Can It Be Now?  2022       4      3
13         Who Can It Be Now?  2011       3      3
14         Who Can It Be Now?  2019       4      2
15         Who Can It Be Now?  2018       4      2
16         Who Can It Be Now?  2005       3      2
17         Who Can It Be Now?  2021       4      1
18         Who Can It Be Now?  2017       4      1
19         Who Can It Be Now?  2016       4      1
20         Who Can It Be Now?  2015       4      1
21         Who Can It Be Now?  2013       4      1
22         Who Can It Be Now?  2004       3      1
23         Who Can It Be Now?  2003       3      1
24         Who Can It Be Now?  1983       1      1
25         Who Can It Be Now?  1982       1      1
26         Who Can It Be Now?  1981       1      1
27         Who Can It Be Now?  1996       2      1

<h3>Pivot by Year</h3>

In [11]:
df_covers_by_year = df_covers_count.pivot_table(index="song",columns="year",values="count")
df_covers_by_year

year                       1981  1982  1983  1996  2000  2003  2004  2005  \
song                                                                        
Be Good Johnny              1.0   NaN   NaN   NaN   NaN   1.0   NaN   1.0   
Down by the Sea             1.0   NaN   NaN   NaN   NaN   NaN   NaN   1.0   
I Can See It in Your Eyes   1.0   NaN   NaN   NaN   1.0   NaN   NaN   NaN   
Underground                 1.0   NaN   NaN   NaN   NaN   NaN   NaN   1.0   
Who Can It Be Now?          1.0   1.0   1.0   1.0   NaN   1.0   1.0   2.0   

year                       2010  2011  2013  2014  2015  2016  2017  2018  \
song                                                                        
Be Good Johnny              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Down by the Sea             NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
I Can See It in Your Eyes   1.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Underground                 NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Who Can It Be Now?          NaN   3.0   1.0   4.0   1.0   1.0   1.0   2.0   

year                       2019  2021  2022  
song                                         
Be Good Johnny              NaN   NaN   1.0  
Down by the Sea             NaN   NaN   NaN  
I Can See It in Your Eyes   NaN   NaN   NaN  
Underground                 NaN   NaN   NaN  
Who Can It Be Now?          2.0   1.0   3.0

<h3>Pivot by Decade</h3>

In [12]:
df_covers_by_decade = df_covers_count.pivot_table(index="song",columns="decade",values="count",aggfunc="sum")
df_covers_by_decade

decade                       1    2    3     4
song                                          
Be Good Johnny             1.0  NaN  2.0   1.0
Down by the Sea            1.0  NaN  1.0   NaN
I Can See It in Your Eyes  1.0  1.0  1.0   NaN
Underground                1.0  NaN  1.0   NaN
Who Can It Be Now?         3.0  1.0  7.0  16.0

<h3>data cleaning</h3>

In [13]:
#subtract 1 from release year because this counts the original release
#df_covers_by_year[release_year] = df_covers_by_year[release_year] -1
# df_covers_by_decade[1] = df_covers_by_decade[1] -1
#df_covers_by_year

<h3>export</h3>

In [14]:
filename_year = f"{bandname}_covers_by_year.csv"
filename_decade = f"{bandname}_covers_by_decade.csv"
df_covers_by_year.to_csv(filename_year)
df_covers_by_decade.to_csv(filename_decade)